# 2. Modeling

### 2.0 Load Train, Val, Test (Numpy Array)

- Gray Scale : Data_gray 폴더 안에 .npy 파일이 있으면 된다.
- RGB Scale : Data 폴더 안에 .npy 파일이 있으면 된다.

#### Scale 설정
- scale = 'gray'
- scale = 'rgb'

In [1]:
scale = 'rgb'
data_dir = 'Data_class_integrate_4'
channel = 3

In [2]:
!ls {data_dir}

X_test.npy  X_train.npy  X_val.npy  y_test.npy	y_train.npy  y_val.npy


#### Load Data

In [3]:
import os
import numpy as np

if os.path.exists(data_dir):
    X_train = np.load(data_dir + '/X_train.npy')
    y_train = np.load(data_dir + '/y_train.npy')
    X_val = np.load(data_dir + '/X_val.npy')
    y_val = np.load(data_dir + '/y_val.npy')
    X_test = np.load(data_dir + '/X_test.npy')
    y_test = np.load(data_dir + '/y_test.npy')

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)
    print(X_test.shape, y_test.shape)

(300, 64, 128, 128, 3) (300, 4)
(100, 64, 128, 128, 3) (100, 4)
(100, 64, 128, 128, 3) (100, 4)


### 2.1 CNN + LSTM

#### 모델 이름 설정

In [10]:
model_name = 'cnn_lstm_class_integrate'

#### Parameters

In [11]:
epochs = 10
batch_size = 2

### 2.1.1 Model Define

In [12]:
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation = 'relu'), input_shape = (64, 128, 128, channel)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3), activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(256))    
model.add(Dense(4, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_7 (TimeDist (None, 64, 126, 126, 32)  896       
_________________________________________________________________
time_distributed_8 (TimeDist (None, 64, 63, 63, 32)    0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 64, 61, 61, 64)    18496     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 64, 30, 30, 64)    0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 64, 28, 28, 128)   73856     
_________________________________________________________________
time_distributed_12 (TimeDis (None, 64, 14, 14, 128)   0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 64, 25088)        

### 2.1.2 Model Compile

In [13]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

### 2.1.3 Model Fit

In [ ]:
%%time

History = model.fit(X_train, y_train,
                   epochs = epochs,
                   batch_size = batch_size,
                   validation_data = (X_val, y_val))

Epoch 1/10
150/150 [==============================] - 541s 4s/step - loss: 1.8715 - accuracy: 0.2739 - val_loss: 1.3594 - val_accuracy: 0.2900
Epoch 2/10
150/150 [==============================] - 539s 4s/step - loss: 1.4048 - accuracy: 0.3457 - val_loss: 1.4069 - val_accuracy: 0.2900
Epoch 3/10
150/150 [==============================] - 540s 4s/step - loss: 1.3859 - accuracy: 0.2841 - val_loss: 1.3727 - val_accuracy: 0.3000
Epoch 4/10
150/150 [==============================] - 541s 4s/step - loss: 1.3977 - accuracy: 0.3099 - val_loss: 1.3725 - val_accuracy: 0.2900
Epoch 5/10
150/150 [==============================] - 560s 4s/step - loss: 1.3964 - accuracy: 0.3028 - val_loss: 1.4284 - val_accuracy: 0.3100
Epoch 6/10
150/150 [==============================] - 604s 4s/step - loss: 1.3992 - accuracy: 0.3030 - val_loss: 1.3747 - val_accuracy: 0.3000
Epoch 7/10
150/150 [==============================] - 765s 5s/step - loss: 1.4182 - accuracy: 0.2333 - val_loss: 1.3918 - val_accuracy: 0.3100

### 2.1.4 Fitted Model Visualization

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(History.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, History.history['loss'])
plt.plot(epochs, History.history['val_loss'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(History.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, History.history['accuracy'])
plt.plot(epochs, History.history['val_accuracy'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.grid()
plt.show()

### 2.1.5 Model Evaluate

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test,
                                batch_size = batch_size)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

### 2.1.6 Model Save

In [ ]:
save_dir = 'Models'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

final_model_name = '/' + scale + '_' + model_name + '.h5'

model.save(save_dir + final_model_name)

### 2.1.7 Saved Model Test

In [ ]:
from keras.models import load_model

rgb_cnn_lstm = load_model(save_dir + final_model_name)

loss, accuracy = rgb_cnn_lstm.evaluate(X_test, y_test,
                                batch_size = batch_size)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))